## Monitoring and visualising model activation traces

In this notebook, we will use RL to train a network on a simple task, and we will visualise the activation traces of the neurons in the network.

### Task

We will use the [AnnubesEnv](https://github.com/neurogym/neurogym/blob/dev/neurogym/envs/annubes.py) environment for this demo. We set the duration of each trial period (fixation, stimulus, decision) and wrap the environment with a Monitor wrapper, which can be used to keep track of activation traces. After training, we can evaluate the agent and visualise the recorded traces. 

In [ ]:
# Ignore warnings.
import warnings

warnings.filterwarnings("ignore")

import numpy as np

from stable_baselines3.common.env_checker import check_env

from sb3_contrib.common.recurrent.policies import RecurrentActorCriticPolicy
from sb3_contrib import RecurrentPPO

from neurogym.envs.native.annubes import AnnubesEnv
from neurogym.wrappers.monitor import Monitor

In [ ]:
# Time step duration in ms
dt = 10

env = AnnubesEnv(dt=dt)

# check the custom environment and output additional warnings (if any)
check_env(env)

# check the environment with a random agent
obs, info = env.reset()
n_steps = 10
for _ in range(n_steps):
    # random action
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated:
        obs, info = env.reset()

print(env.timing)
print("----------------")
print(env.observation_space)
print(env.observation_space.name)
print("----------------")
print(env.action_space)
print(env.action_space.name)

Create a recurrent model and show the policy.

In [ ]:
# Create the agent that will be trained in this environment
model = RecurrentPPO(
    RecurrentActorCriticPolicy, env, policy_kwargs={"lstm_hidden_size": 64}
)

model.policy

Get a reference to the action net from the policy

In [ ]:
action_net = model.policy.get_submodule("action_net")

Make sure that the time step is correctly registered

In [ ]:
env.dt

### Train the agent

The agent can be trained on different environments. calls both the actor and the critic networks, so if the activations of the value net are needed, now is the opportunity to record them.

In [ ]:
# Create an environment for evaluation
annubes_env = AnnubesEnv(dt=dt)

# Create a monitor
env = Monitor(env, name=f"NeuroGym Monitor | {annubes_env.__class__.__qualname__}")


Register the action net (a single layer in our case) with the monitor using the `record_activations()` method. The activations will be automatically extracted and stored after each step only for that layer. You can add as many layers as necessary in this way.

In [ ]:
am = env.record_activations(action_net, "Action net")

Check the shape of the output of this layer.

In [ ]:
am.monitor.get_output_shape()

Train the agent for a number of steps.

In [ ]:
# Set the number timesteps for training
total_timesteps = 1000

# Train the agent.
model.learn(total_timesteps=total_timesteps)

Check out the shape of a single trace

In [ ]:
am.history[0].shape

Evaluate the agent over a number of trials

In [ ]:
n_trials = 10

# Evaluate the policy
env.evaluate_policy(n_trials, model)

Show the length of the history. Should be equal to the number of trials.

In [ ]:
len(am.history)

Plot the activations in a separate panel for each neuron.

In [ ]:
fig, axes = env.plot_activations(am.name)

Plot the __mean__ activations in a separate panel for each neuron.

In [ ]:
fig_mean, axes_mean = env.plot_activations(am.name, mean = True)